# Bibliotecas

In [ ]:
!pip install praw
!pip install -U deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from numba import jit, prange, njit
import time
from deep_translator import GoogleTranslator, single_detection
import praw
import pandas as pd

# Extração

In [ ]:
reddit_read_only = praw.Reddit(client_id="dOLvr35kk_81cIumKlA8yQ",
                               client_secret="Z1iMBY4HN_MWshEPU9GoqRGhtTlN9A",
                               user_agent="Old-Ad9688")

In [ ]:
subreddit = reddit_read_only.subreddit("wine")

# Obtenha todos os posts do subreddit
posts = subreddit.new(limit=None)

posts_dict = {
    "Title": [],
    "Post Text": [],
    "ID": [],
    "Score": [],
    "Total Comments": [],
    "Post URL": [],
    "Comments": []  # Lista de comentários associados a cada postagem
}

for post in posts:
    # Título de cada post
    posts_dict["Title"].append(post.title)

    # Texto dentro do post, tratando texto ausente
    if post.selftext:
        posts_dict["Post Text"].append(post.selftext)
    else:
        posts_dict["Post Text"].append("")

    # ID único de cada post
    posts_dict["ID"].append(post.id)

    # A pontuação de um post
    posts_dict["Score"].append(post.score)

    # Total de comentários dentro do post
    posts_dict["Total Comments"].append(post.num_comments)

    # URL de cada post
    posts_dict["Post URL"].append(post.url)

    # Obter os comentários da postagem
    comments = []
    post.comments.replace_more(limit=None)  # Expandir todos os comentários
    for comment in post.comments.list():
        comments.append(comment.body)
    posts_dict["Comments"].append(comments)

# Salvar os dados em um DataFrame do pandas
top_posts = pd.DataFrame(posts_dict)
print(top_posts)
pd.DataFrame(posts_dict)
top_posts

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

KeyboardInterrupt: ignored

In [ ]:
top_posts.to_excel("Top Posts.xlsx", index=True)

In [ ]:
top_posts = pd.read_excel("Top Posts.xlsx")

# Tradução

In [ ]:
def split_text(texto, limite):
    texto = texto.strip()

    if len(texto) <= limite:
        return [texto]

    palavras = texto.split()
    partes, parte_atual = [], ""

    for palavra in palavras:
        if len(parte_atual) + len(palavra) + 1 <= limite:
            parte_atual += palavra + " "
        else:
            partes.append(parte_atual.strip())
            parte_atual = palavra + " "

    partes.append(parte_atual.strip())

    return partes

In [ ]:
# traduzindo dados para português
@jit(parallel=True)
def translate_term(term):
  translated_list = []
  my_translator = GoogleTranslator(source='auto', target='portuguese')

  for c in prange(len(term)):
    text = str(term[c])
    if isinstance(text, str) and not text.isdigit():
      result_split = []
      texts = split_text(text, 5000)

      for s_text in texts:
        result_split.append(my_translator.translate(text=s_text))

      result = ' '.join(result_split) if len(result_split) > 1 else result_split

    else:
      result = text

    translated_list.append(result)

  return translated_list

def translate_cols(df, cols):
  for c in cols:
    df[c] = translate_term(df[c].values)
    print(f'column {c} translated!')
  return df

In [ ]:
cols_to_translate = ['Comments', 'Title', 'Post Text']

tempo_inicial = time.time()
translated_top_posts = translate_cols(top_posts, cols_to_translate)
tempo_final = time.time()
print(f"A duração da tradução foi de {round((tempo_final - tempo_inicial)/60, 2)} minutos")

column Comments translated!
column Title translated!
column Post Text translated!
A duração da tradução foi de 47.63 minutos


In [ ]:
translated_top_posts.to_csv('translated_top_posts.csv', index=False)

# Tratamento

In [1]:
import pandas as pd
import re
import os

In [5]:
def clear_text(text):
  text = '' if 'Obrigado por sua submissão ao' in text else text
  clean_text = re.sub(r"(\[\'em\'\])", '', text)
  clean_text = re.sub(r"(\[None\])", '', clean_text)
  clean_text = re.sub(r"(\[\'|\'\])", '', clean_text)
  clean_text = re.sub(r"(\[\"|\"\])", '', clean_text)
  clean_text = re.sub(r"(\[\\'|\\'\])", '', clean_text)

  return clean_text

In [6]:
def save_in_txt(df, cols):
  with open('reddit.txt', 'w', encoding='utf-8') as f:
    for col in cols:
      for texto in df[col]:
          f.write(texto + '\n')

In [7]:
df = pd.read_csv('translated_reddit_top_posts.csv')
cols = ['Title', 'Post Text', 'Comments']

for c in cols:
  df[c] = df[c].apply(clear_text)

save_in_txt(df, cols)